In [ ]:
%cd ..

import torch
import os
from PIL import Image

from helpers.models.hidream.pipeline import HiDreamImagePipeline
from helpers.models.hidream.transformer import HiDreamImageTransformer2DModel
from transformers import PreTrainedTokenizerFast, LlamaForCausalLM
from lycoris import create_lycoris_from_weights

# --------------------------
# Load Model + Apply LoRA
# --------------------------

llama_repo = "/local/yada/models/Meta-Llama-3.1-8B-Instruct"
model_id = "HiDream-ai/HiDream-I1-Full"
lora_path = "/local/yada/apps/SimpleTuner-a/output/models-hidream-run04/pytorch_lora_weights.safetensors"

tokenizer_4 = PreTrainedTokenizerFast.from_pretrained(llama_repo)
text_encoder_4 = LlamaForCausalLM.from_pretrained(
    llama_repo, output_hidden_states=True, torch_dtype=torch.bfloat16
)
transformer = HiDreamImageTransformer2DModel.from_pretrained(
    model_id, torch_dtype=torch.bfloat16, subfolder="transformer"
)

pipeline = HiDreamImagePipeline.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    tokenizer_4=tokenizer_4,
    text_encoder_4=text_encoder_4,
    transformer=transformer,
)

device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
pipeline.to(device)

wrapper, _ = create_lycoris_from_weights(1.0, lora_path, pipeline.transformer)
wrapper.merge_to()

# --------------------------
# Generation Function
# --------------------------

def generate_images(
    prompts,
    negative_prompt,
    width,
    height,
    guidance_scale,
    num_inference_steps,
    seed,
    save_dir
):
    os.makedirs(save_dir, exist_ok=True)
    
    t5_embeds, llama_embeds, negative_t5_embeds, negative_llama_embeds, pooled_embeds, negative_pooled_embeds = pipeline.encode_prompt(
        prompt=prompts,
        prompt_2=prompts,
        prompt_3=prompts,
        prompt_4=prompts,
        num_images_per_prompt=1
    )

    images = pipeline(
        t5_prompt_embeds=t5_embeds,
        llama_prompt_embeds=llama_embeds,
        pooled_prompt_embeds=pooled_embeds,
        negative_t5_prompt_embeds=negative_t5_embeds,
        negative_llama_prompt_embeds=negative_llama_embeds,
        negative_pooled_prompt_embeds=negative_pooled_embeds,
        num_inference_steps=num_inference_steps,
        generator=torch.Generator(device=device).manual_seed(seed),
        width=width,
        height=height,
        guidance_scale=guidance_scale,
    ).images

    for i, img in enumerate(images):
        img.save(os.path.join(save_dir, f"image_{i+1}.png"))

    return images

/root/miniconda3/envs/simpletuner/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/local/yada/apps/SimpleTuner-a
[2025-04-21 18:54:09,681] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/root/miniconda3/envs/simpletuner/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/root/miniconda3/envs/simpletuner/compiler_compat/ld: warning: librt.so.1, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/envs/simpletuner/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/envs/simpletuner/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/envs/simpletuner/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/envs/simpletuner/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/root/miniconda3/envs/simp

HiDream: FlashAttention3 not found, trying FlashAttention2
HiDream: FlashAttention2 failed to load (No module named 'flash_attn'), falling back to Torch SDPA.


/root/miniconda3/envs/simpletuner/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:820: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output_hidden_states` is. When `return_dict_in_generate` is not `True`, `output_hidden_states` is ignored.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/11 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
prompts = [
    "a girl with silver hair, detailed background",
    "a warrior in armor standing on a cliff",
    "a futuristic city at night with neon lights",
    "a serene landscape with cherry blossoms"
]
negative_prompt = "blurry, distorted"
width, height = 768, 1024
guidance_scale = 5.0
num_inference_steps = 28
seed = 123456
save_dir = "output/oneshot_batch"

res = generate_images(
    prompts=prompts,
    negative_prompt=negative_prompt,
    width=width,
    height=height,
    guidance_scale=guidance_scale,
    num_inference_steps=num_inference_steps,
    seed=seed,
    save_dir=save_dir
)

In [ ]:
res

In [ ]:
from procslib import get_model, print_model_keys

print_model_keys("aes")